In [ ]:
import json
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import re
import os
import sys
import javalang
from xml.dom.minidom import parse
import xml.dom.minidom

In [ ]:
def variable_query(root, name, target_list):
    children = None
    if root is None:
        #print(root)
        return 1
    
    if isinstance(root, javalang.ast.Node):
        children = root.children
    elif isinstance(root, (list, tuple)):
        children = root
    else:
        return
    for child in children:
        if isinstance(child, (javalang.tree.VariableDeclarator)) and child.name == name:
            target_list.append(child)
        if isinstance(child, (javalang.ast.Node, list, tuple)):
            variable_query(child,name,target_list)

            
def permission_check(permission_list,file_path):
    try:
        DOMTree = xml.dom.minidom.parse(file_path)
    except:
        return False
    collection = DOMTree.documentElement
    permission_tags = collection.getElementsByTagName('uses-permission')
    for tag in permission_tags:
        name = tag.getAttribute("android:name")
        if name in permission_list:
            return False
    return True
def factors_check(name_list,text):
    for name in name_list:
        if name in text:
            return True
    return False

In [ ]:
def walk_tree_hardware(root,violation_list,var_names,avtivity_flag):
    children = None
    if root is None:
        #print(root)
        return 
    
    if isinstance(root, javalang.ast.Node):
        children = root.children
    elif isinstance(root, (list, tuple)):
        children = root
    else:
        return
    
    for child in children:
            
            
        if (violation_list) == 0 and isinstance(child, (javalang.tree.ClassDeclaration)):
            avtivity_flag = 1
        
        if isinstance(child, (javalang.tree.FieldDeclaration,javalang.tree.LocalVariableDeclaration)):
            if ('Camera' in child.type.name or 'CameraDevice' in child.type.name):
                for declarator in child.declarators:
                    if isinstance(declarator, (javalang.tree.VariableDeclarator)):
                        var_names.append(declarator.name)
                        
        if isinstance(child, javalang.tree.This):
            for member in child.selectors:
                if isinstance(member, (javalang.tree.MethodInvocation)):
                    #print(member.member)
                    if isinstance(child.selectors[0], javalang.tree.MemberReference):
                        if child.selectors[0].member in var_names and \
                        (member.member == 'takePicture' or member.member == 'createCaptureSession'):
                            violation_list.append('violation')
        
        if isinstance(child, (javalang.tree.MethodInvocation)):
            #print('method ',child.member)
            #print('qualifier ',child.qualifier)
            #if child.member == 'takePicture' or child.member == 'createCaptureSession':
            #    violation_list.append('violation')
            if child.qualifier in var_names and (child.member == 'takePicture' or child.member == 'createCaptureSession'):
                violation_list.append('violation')
            
        if isinstance(child, (javalang.ast.Node, list, tuple)):
            walk_tree_hardware(child,violation_list,var_names,avtivity_flag)


In [ ]:
def view_security(file_path):
    try:
        DOMTree = xml.dom.minidom.parse(file_path)
    except:
        return False
    collection = DOMTree.documentElement
    view_tags = collection.getElementsByTagName('SurfaceView')
    for tag in view_tags:
        width = tag.getAttribute("android:layout_width")
        height = tag.getAttribute("android:layout_height")
        if (width.isnumeric() and width < 10) or (height.isnumeric() and height < 10):
            return True
    return False

In [ ]:
def hardware_security_detecor_xml(file,root):
    if file.endswith(".xml"):
        file_path = os.path.join(root, file)
        if view_security(file_path):
            print('hardware_security violation')
            return 1
    return 0
        
def hardware_security_detecor(file,root,text,treenode):
    
    if text == False and treenode == False :
        return 0

    factor_list = ['Camera', 'takePicture', 'createCaptureSession']
    if not factors_check(factor_list, text):
        return 0

    violation_list = list()
    name_list = list()
    avtivity_flag = 0
    walk_tree_hardware(treenode,violation_list,name_list,avtivity_flag)
    if avtivity_flag and 'violation' in violation_list:
        print('hardware_security violation')
        return 0
    return 0

In [ ]:
i=0

pridir_code = 'D:/jupyterCode/summer_object/apks/decomplied/'
for code_folder in os.listdir(pridir_code)[:]:
    #with open('text.txt','r') as file:
    #    code_folder = file.read()
    if code_folder.endswith(".zip"):
        continue
        
    row = {'apk_name':code_folder,'mtp_SD':0,'hardware_security':0,'nfc_SD':0,'animation_hedonism':0, \
           'media_SD':0,'media_hedonism':0,'media_Uni':0,'nfc_security':0,\
       'telephony_security':0,'telephony_conformity':0}
    full_code_folder = pridir_code+code_folder
    for root, dirs, files in os.walk(full_code_folder):
        for file in files:

            try:
                if file.endswith(".java"):
                    file_path = os.path.join(root, file)

                #hardware
                if not row['hardware_security']:
                    row['hardware_security'] = hardware_security_detecor_xml(file,root)

                    try:
                        with open(file_path, "r") as file:
                            text = file.read()
                        treenode = javalang.parse.parse(text)
                    except:
                        text = False
                        treenode = False
                        continue
                    #animation
                    if not row['hardware_security']:
                        row['hardware_security'] = hardware_security_detecor(file,root,text,treenode)


            except:
                print('error')
    i+=1